In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats 

sns.set_theme()

In [2]:
data=pd.read_csv('../data/experienceAnalysis.csv')

In [19]:
data

,MSISDN/Number,Engagement Cluster,Handset Type,IMEI,Last Location Name,IMSI,Handset Manufacturer,Bearer Id,Dur. (ms).1,Avg RTT DL (ms),...,Netflix Total (Bytes),Gaming Total (Bytes),Other Total (Bytes),Avg Bearer TP DL (Mbps),Avg Bearer TP UL (Mbps),Activity Duration DL (s),Activity Duration UL (s),Total UL+DL (Bytes),Total Session (Mbs),Experience Cluster
0,3.360100e+10,0,Huawei P20 Lite Huawei Nova 3E,8.627420e+13,L74538A,2.082014e+14,Huawei,1,1167.20,46.000000,...,27180981.0,812458661.0,386570872.0,0.036133,0.038086,26.588,33.662,878690574.0,1197.612347,0
1,3.360100e+10,0,Apple iPhone 7 (A1778),3.553261e+13,D12741B,2.082015e+14,Apple,1,1812.30,30.000000,...,11221763.0,119750078.0,281710071.0,0.046875,0.049805,49.283,54.751,156859643.0,416.200012,0
2,3.360100e+10,0,undefined,3.561271e+13,L96414B,2.082019e+14,undefined,1,1349.69,109.795706,...,19353900.0,538827713.0,501693672.0,0.046875,0.047852,16.793,18.434,595966483.0,1040.509209,0
3,3.360101e+10,0,Apple iPhone 5S (A1457),3.520491e+13,D93721A,2.082017e+14,Apple,1,498.78,69.000000,...,1942092.0,391126127.0,35279702.0,0.199219,0.042969,12.097,4.497,422320698.0,426.834773,1
4,3.360101e+10,0,Samsung Galaxy A8 (2018),3.584911e+13,T10318C,2.082017e+14,Samsung,1,1280.88,21.000000,...,21735594.0,126130650.0,212253545.0,7.288086,0.316406,52.678,49.457,194367933.0,377.202128,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80218,3.378996e+10,0,Samsung Galaxy S8 Plus (Sm-G955F),3.578211e+13,D74791B,2.082017e+14,Samsung,1,1570.69,109.795706,...,24293718.0,638609868.0,712062086.0,0.040039,0.063477,34.172,36.809,710287575.0,1350.761025,0
80219,3.378997e+10,0,Apple iPhone 6S (A1688),3.591551e+13,T93758B,2.082015e+14,Apple,1,1604.61,48.000000,...,30370302.0,134815919.0,408107210.0,0.022461,0.042969,35.138,48.886,208123067.0,587.675828,0
80220,3.379000e+10,0,Huawei Honor 9 Lite,8.687980e+13,T93842C,2.082014e+14,Huawei,1,88.10,42.000000,...,14902538.0,660361359.0,295282817.0,9.744141,0.377930,259.862,241.044,714641641.0,959.427761,1
80221,3.379000e+10,0,Apple iPhone 8 Plus (A1897),3.529761e+13,D42155B,2.082010e+14,Apple,1,1409.88,34.000000,...,8531060.0,437003292.0,211115113.0,0.066406,0.046875,33.435,39.006,480307335.0,653.212936,0


In [3]:
# No of xDR sessions
aggrData=data.groupby(['MSISDN/Number']).agg({'Dur. (ms).1':sum,'Total Session (Mbs)':sum,'Bearer Id':'count'})
aggrData=aggrData.rename(columns={'Bearer Id': 'No Sessions'})
aggrData

,Dur. (ms).1,Total Session (Mbs),No Sessions
MSISDN/Number,,,
3.360100e+10,1167.20,1197.612347,1
3.360100e+10,1812.30,416.200012,1
3.360100e+10,1349.69,1040.509209,1
3.360101e+10,498.78,426.834773,1
3.360101e+10,2539.83,1044.822530,2
...,...,...,...
3.378996e+10,1570.69,1350.761025,1
3.378997e+10,1604.61,587.675828,1
3.379000e+10,88.10,959.427761,1


In [4]:
skewedData=aggrData[['Dur. (ms).1','Total Session (Mbs)','No Sessions']]

In [5]:
skewedData

,Dur. (ms).1,Total Session (Mbs),No Sessions
MSISDN/Number,,,
3.360100e+10,1167.20,1197.612347,1
3.360100e+10,1812.30,416.200012,1
3.360100e+10,1349.69,1040.509209,1
3.360101e+10,498.78,426.834773,1
3.360101e+10,2539.83,1044.822530,2
...,...,...,...
3.378996e+10,1570.69,1350.761025,1
3.378997e+10,1604.61,587.675828,1
3.379000e+10,88.10,959.427761,1


In [6]:
from sklearn.cluster import KMeans
Kmean = KMeans(n_clusters=3)

Kmean.fit(skewedData)
distances=Kmean.cluster_centers_

In [9]:
dist=distances[-1:]

In [10]:
eucDis=np.linalg.norm(skewedData-dist,axis=1).reshape(-1,1)
eucDis

array([[ 516.49704768],
       [ 546.11815389],
       [ 280.38756969],
       ...,
       [1515.58534638],
       [ 330.02601915],
       [ 951.56993924]])

In [16]:
skewedData['Engagement Score']=eucDis

In [17]:
skewedData

,Dur. (ms).1,Total Session (Mbs),No Sessions,Engagement Socre,Engagement Score
MSISDN/Number,,,,,
3.360100e+10,1167.20,1197.612347,1,516.497048,516.497048
3.360100e+10,1812.30,416.200012,1,546.118154,546.118154
3.360100e+10,1349.69,1040.509209,1,280.387570,280.387570
3.360101e+10,498.78,426.834773,1,1209.802972,1209.802972
3.360101e+10,2539.83,1044.822530,2,944.828816,944.828816
...,...,...,...,...,...
3.378996e+10,1570.69,1350.761025,1,431.289340,431.289340
3.378997e+10,1604.61,587.675828,1,333.025197,333.025197
3.379000e+10,88.10,959.427761,1,1515.585346,1515.585346


In [23]:
skewed=skewedData.reset_index()
skewed=skewed[['MSISDN/Number','Engagement Score']]
data=data.merge(skewed,on='MSISDN/Number')
data

,MSISDN/Number,Engagement Cluster,Handset Type,IMEI,Last Location Name,IMSI,Handset Manufacturer,Bearer Id,Dur. (ms).1,Avg RTT DL (ms),...,Gaming Total (Bytes),Other Total (Bytes),Avg Bearer TP DL (Mbps),Avg Bearer TP UL (Mbps),Activity Duration DL (s),Activity Duration UL (s),Total UL+DL (Bytes),Total Session (Mbs),Experience Cluster,Engagement Score
0,3.360100e+10,0,Huawei P20 Lite Huawei Nova 3E,8.627420e+13,L74538A,2.082014e+14,Huawei,1,1167.20,46.000000,...,812458661.0,386570872.0,0.036133,0.038086,26.588,33.662,878690574.0,1197.612347,0,516.497048
1,3.360100e+10,0,Apple iPhone 7 (A1778),3.553261e+13,D12741B,2.082015e+14,Apple,1,1812.30,30.000000,...,119750078.0,281710071.0,0.046875,0.049805,49.283,54.751,156859643.0,416.200012,0,546.118154
2,3.360100e+10,0,undefined,3.561271e+13,L96414B,2.082019e+14,undefined,1,1349.69,109.795706,...,538827713.0,501693672.0,0.046875,0.047852,16.793,18.434,595966483.0,1040.509209,0,280.387570
3,3.360101e+10,0,Apple iPhone 5S (A1457),3.520491e+13,D93721A,2.082017e+14,Apple,1,498.78,69.000000,...,391126127.0,35279702.0,0.199219,0.042969,12.097,4.497,422320698.0,426.834773,1,1209.802972
4,3.360101e+10,0,Samsung Galaxy A8 (2018),3.584911e+13,T10318C,2.082017e+14,Samsung,1,1280.88,21.000000,...,126130650.0,212253545.0,7.288086,0.316406,52.678,49.457,194367933.0,377.202128,1,944.828816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80218,3.378996e+10,0,Samsung Galaxy S8 Plus (Sm-G955F),3.578211e+13,D74791B,2.082017e+14,Samsung,1,1570.69,109.795706,...,638609868.0,712062086.0,0.040039,0.063477,34.172,36.809,710287575.0,1350.761025,0,431.289340
80219,3.378997e+10,0,Apple iPhone 6S (A1688),3.591551e+13,T93758B,2.082015e+14,Apple,1,1604.61,48.000000,...,134815919.0,408107210.0,0.022461,0.042969,35.138,48.886,208123067.0,587.675828,0,333.025197
80220,3.379000e+10,0,Huawei Honor 9 Lite,8.687980e+13,T93842C,2.082014e+14,Huawei,1,88.10,42.000000,...,660361359.0,295282817.0,9.744141,0.377930,259.862,241.044,714641641.0,959.427761,1,1515.585346
80221,3.379000e+10,0,Apple iPhone 8 Plus (A1897),3.529761e+13,D42155B,2.082010e+14,Apple,1,1409.88,34.000000,...,437003292.0,211115113.0,0.066406,0.046875,33.435,39.006,480307335.0,653.212936,0,330.026019


In [21]:
skewed=skewedData.reset_index()

,MSISDN/Number,Dur. (ms).1,Total Session (Mbs),No Sessions,Engagement Socre,Engagement Score
0,3.360100e+10,1167.20,1197.612347,1,516.497048,516.497048
1,3.360100e+10,1812.30,416.200012,1,546.118154,546.118154
2,3.360100e+10,1349.69,1040.509209,1,280.387570,280.387570
3,3.360101e+10,498.78,426.834773,1,1209.802972,1209.802972
4,3.360101e+10,2539.83,1044.822530,2,944.828816,944.828816
...,...,...,...,...,...,...
73369,3.378996e+10,1570.69,1350.761025,1,431.289340,431.289340
73370,3.378997e+10,1604.61,587.675828,1,333.025197,333.025197
73371,3.379000e+10,88.10,959.427761,1,1515.585346,1515.585346
73372,3.379000e+10,1409.88,653.212936,1,330.026019,330.026019


In [25]:
reportData=data.groupby(['MSISDN/Number','Handset Type']).agg({'Avg Bearer TP UL (kbps)':np.mean,'Avg Bearer TP DL (kbps)':np.mean,'TCP DL Retrans. Vol (Bytes)':np.mean,'TCP UL Retrans. Vol (Bytes)':np.mean})

In [26]:
from sklearn.cluster import KMeans
Kmean = KMeans(n_clusters=3)

Kmean.fit(reportData)
distances=Kmean.cluster_centers_

In [28]:
distances

array([[9.39103549e+01, 6.43002400e+02, 2.08480150e+07, 7.34880902e+05],
       [8.76497214e+02, 1.35319332e+04, 1.34493099e+06, 2.92959931e+05],
       [1.34636000e+03, 1.28184000e+04, 1.32542452e+08, 3.59755139e+05]])

In [29]:
dist=distances[-1:]
eucDis=np.linalg.norm(reportData-dist,axis=1).reshape(-1,1)
eucDis

array([[1.11733254e+08],
       [1.11733254e+08],
       [1.11733254e+08],
       ...,
       [1.32327889e+08],
       [1.11733254e+08],
       [1.11733254e+08]])

In [30]:
reportData['Experience Score']=eucDis
reportData


Avg Bearer TP UL (kbps)  \
MSISDN/Number Handset Type                                                 
3.360100e+10  Huawei P20 Lite Huawei Nova 3E                        39.0   
              Apple iPhone 7 (A1778)                                51.0   
              undefined                                             49.0   
3.360101e+10  Apple iPhone 5S (A1457)                               44.0   
              Samsung Galaxy A8 (2018)                             193.0   
...                                                                  ...   
3.378996e+10  Samsung Galaxy S8 Plus (Sm-G955F)                     65.0   
3.378997e+10  Apple iPhone 6S (A1688)                               44.0   
3.379000e+10  Huawei Honor 9 Lite                                  387.0   
              Apple iPhone 8 Plus (A1897)                           48.0   
3.370000e+14  Huawei B525S-23A                                      22.0   

                                                 Avg Bearer TP DL (kbps)  \
MSISDN/Number Handset Type                                                 
3.360100e+10  Huawei P20 Lite Huawei Nova 3E                        37.0   
              Apple iPhone 7 (A1778)                                48.0   
              undefined                                             48.0   
3.360101e+10  Apple iPhone 5S (A1457)                              204.0   
              Samsung Galaxy A8 (2018)                            3761.0   
...                                                                  ...   
3.378996e+10  Samsung Galaxy S8 Plus (Sm-G955F)                     41.0   
3.378997e+10  Apple iPhone 6S (A1688)                               23.0   
3.379000e+10  Huawei Honor 9 Lite                                 9978.0   
              Apple iPhone 8 Plus (A1897)                           68.0   
3.370000e+14  Huawei B525S-23A                                      11.0   

                                                 TCP DL Retrans. Vol (Bytes)  \
MSISDN/Number Handset Type                                                     
3.360100e+10  Huawei P20 Lite Huawei Nova 3E                    2.080991e+07   
              Apple iPhone 7 (A1778)                            2.080991e+07   
              undefined                                         2.080991e+07   
3.360101e+10  Apple iPhone 5S (A1457)                           1.066000e+03   
              Samsung Galaxy A8 (2018)                          1.040634e+07   
...                                                                      ...   
3.378996e+10  Samsung Galaxy S8 Plus (Sm-G955F)                 2.080991e+07   
3.378997e+10  Apple iPhone 6S (A1688)                           2.080991e+07   
3.379000e+10  Huawei Honor 9 Lite                               2.150440e+05   
              Apple iPhone 8 Plus (A1897)                       2.080991e+07   
3.370000e+14  Huawei B525S-23A                                  2.080991e+07   

                                                 TCP UL Retrans. Vol (Bytes)  \
MSISDN/Number Handset Type                                                     
3.360100e+10  Huawei P20 Lite Huawei Nova 3E                   759658.664811   
              Apple iPhone 7 (A1778)                           759658.664811   
              undefined                                        759658.664811   
3.360101e+10  Apple iPhone 5S (A1457)                          759658.664811   
              Samsung Galaxy A8 (2018)                         759658.664811   
...                                                                      ...   
3.378996e+10  Samsung Galaxy S8 Plus (Sm-G955F)                759658.664811   
3.378997e+10  Apple iPhone 6S (A1688)                          759658.664811   
3.379000e+10  Huawei Honor 9 Lite                                3001.000000   
              Apple iPhone 8 Plus (A1897)                      759658.664811   
3.370000e+14  Huawei B525S-23A                                 759658.

In [31]:
report=reportData.reset_index()
report=report[['MSISDN/Number','Experience Score']]
data=data.merge(report,on='MSISDN/Number')
data

,MSISDN/Number,Engagement Cluster,Handset Type,IMEI,Last Location Name,IMSI,Handset Manufacturer,Bearer Id,Dur. (ms).1,Avg RTT DL (ms),...,Other Total (Bytes),Avg Bearer TP DL (Mbps),Avg Bearer TP UL (Mbps),Activity Duration DL (s),Activity Duration UL (s),Total UL+DL (Bytes),Total Session (Mbs),Experience Cluster,Engagement Score,Experience Score
0,3.360100e+10,0,Huawei P20 Lite Huawei Nova 3E,8.627420e+13,L74538A,2.082014e+14,Huawei,1,1167.20,46.000000,...,386570872.0,0.036133,0.038086,26.588,33.662,878690574.0,1197.612347,0,516.497048,1.117333e+08
1,3.360100e+10,0,Apple iPhone 7 (A1778),3.553261e+13,D12741B,2.082015e+14,Apple,1,1812.30,30.000000,...,281710071.0,0.046875,0.049805,49.283,54.751,156859643.0,416.200012,0,546.118154,1.117333e+08
2,3.360100e+10,0,undefined,3.561271e+13,L96414B,2.082019e+14,undefined,1,1349.69,109.795706,...,501693672.0,0.046875,0.047852,16.793,18.434,595966483.0,1040.509209,0,280.387570,1.117333e+08
3,3.360101e+10,0,Apple iPhone 5S (A1457),3.520491e+13,D93721A,2.082017e+14,Apple,1,498.78,69.000000,...,35279702.0,0.199219,0.042969,12.097,4.497,422320698.0,426.834773,1,1209.802972,1.325420e+08
4,3.360101e+10,0,Samsung Galaxy A8 (2018),3.584911e+13,T10318C,2.082017e+14,Samsung,1,1280.88,21.000000,...,212253545.0,7.288086,0.316406,52.678,49.457,194367933.0,377.202128,1,944.828816,1.221368e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81092,3.378996e+10,0,Samsung Galaxy S8 Plus (Sm-G955F),3.578211e+13,D74791B,2.082017e+14,Samsung,1,1570.69,109.795706,...,712062086.0,0.040039,0.063477,34.172,36.809,710287575.0,1350.761025,0,431.289340,1.117333e+08
81093,3.378997e+10,0,Apple iPhone 6S (A1688),3.591551e+13,T93758B,2.082015e+14,Apple,1,1604.61,48.000000,...,408107210.0,0.022461,0.042969,35.138,48.886,208123067.0,587.675828,0,333.025197,1.117333e+08
81094,3.379000e+10,0,Huawei Honor 9 Lite,8.687980e+13,T93842C,2.082014e+14,Huawei,1,88.10,42.000000,...,295282817.0,9.744141,0.377930,259.862,241.044,714641641.0,959.427761,1,1515.585346,1.323279e+08
81095,3.379000e+10,0,Apple iPhone 8 Plus (A1897),3.529761e+13,D42155B,2.082010e+14,Apple,1,1409.88,34.000000,...,211115113.0,0.066406,0.046875,33.435,39.006,480307335.0,653.212936,0,330.026019,1.117333e+08


In [32]:
data['Satisfaction score']=(data['Engagement Score']+data['Experience Score'])/2

In [34]:
data.groupby(['MSISDN/Number']).agg({'Satisfaction score':np.mean}).sort_values(by='Satisfaction score',ascending=False)

,Satisfaction score
MSISDN/Number,
3.376557e+10,6.630982e+07
3.368177e+10,6.627525e+07
3.366762e+10,6.627328e+07
3.369988e+10,6.627272e+07
3.369844e+10,6.627233e+07
...,...
3.366830e+10,5.218377e+06
3.362023e+10,4.456798e+06
3.365046e+10,3.807719e+06


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81097 entries, 0 to 81096
Data columns (total 52 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   MSISDN/Number                81097 non-null  float64
 1   Engagement Cluster           81097 non-null  int64  
 2   Handset Type                 81097 non-null  object 
 3   IMEI                         81097 non-null  float64
 4   Last Location Name           81097 non-null  object 
 5   IMSI                         81097 non-null  float64
 6   Handset Manufacturer         81097 non-null  object 
 7   Bearer Id                    81097 non-null  int64  
 8   Dur. (ms).1                  81097 non-null  float64
 9   Avg RTT DL (ms)              81097 non-null  float64
 10  Avg RTT UL (ms)              81097 non-null  float64
 11  Avg Bearer TP DL (kbps)      81097 non-null  float64
 12  Avg Bearer TP UL (kbps)      81097 non-null  float64
 13  TCP DL Retrans. 

In [39]:
dataDF=data.drop(labels=['Handset Manufacturer','Handset Type','IMEI','IMSI','Last Location Name'],axis=1)

In [40]:
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81097 entries, 0 to 81096
Data columns (total 47 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   MSISDN/Number                81097 non-null  float64
 1   Engagement Cluster           81097 non-null  int64  
 2   Bearer Id                    81097 non-null  int64  
 3   Dur. (ms).1                  81097 non-null  float64
 4   Avg RTT DL (ms)              81097 non-null  float64
 5   Avg RTT UL (ms)              81097 non-null  float64
 6   Avg Bearer TP DL (kbps)      81097 non-null  float64
 7   Avg Bearer TP UL (kbps)      81097 non-null  float64
 8   TCP DL Retrans. Vol (Bytes)  81097 non-null  float64
 9   TCP UL Retrans. Vol (Bytes)  81097 non-null  float64
 10  HTTP DL (Bytes)              81097 non-null  float64
 11  HTTP UL (Bytes)              81097 non-null  float64
 12  Activity Duration DL (ms)    81097 non-null  float64
 13  Activity Duratio

In [41]:
X=dataDF.drop(['Satisfaction score'],axis=1)
y=dataDF['Satisfaction score'].values

In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [44]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=3,test_size=.3)

In [51]:
lr=LinearRegression()
rf=RandomForestRegressor()

lr.fit(X_train,y_train)
predictions=lr.predict(X_test)

In [46]:
lr.score(X_train,y_train)

1.0

In [47]:
lr.score(X_test,y_test)

1.0

In [49]:
mean_squared_error(y_test,predictions)

3.559823112753159e-09

In [52]:
rf.fit(X_train,y_train)
predictionsRF=rf.predict(X_test)

In [53]:
mean_squared_error(y_test,predictionsRF)

792441000.6856102

In [56]:
import pickle


pickle.dump(rf,open('../models/rfModel.pkl','wb'))

In [58]:
metricData=data[['Experience Score','Engagement Score']]

In [59]:
metricData

,Experience Score,Engagement Score
0,1.117333e+08,516.497048
1,1.117333e+08,546.118154
2,1.117333e+08,280.387570
3,1.325420e+08,1209.802972
4,1.221368e+08,944.828816
...,...,...
81092,1.117333e+08,431.289340
81093,1.117333e+08,333.025197
81094,1.323279e+08,1515.585346
81095,1.117333e+08,330.026019


In [62]:
from sklearn.cluster import KMeans
Kmean = KMeans(n_clusters=2)

Kmean.fit(metricData)
cluster=Kmean.fit_predict(metricData)

In [64]:
data['Metric Satis']=cluster

In [65]:
data.groupby(['Metric Satis']).agg({'Experience Score':np.mean,'Engagement Score':np.mean})

,Experience Score,Engagement Score
Metric Satis,,
0,1.116934e+08,899.228589
1,1.305785e+08,1264.254752


In [67]:
finalData=data[['MSISDN/Number','Experience Score','Engagement Score','Satisfaction score']]

In [68]:
finalData.to_csv('../data/finalData.csv',index=False)

In [69]:
finalData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81097 entries, 0 to 81096
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MSISDN/Number       81097 non-null  float64
 1   Experience Score    81097 non-null  float64
 2   Engagement Score    81097 non-null  float64
 3   Satisfaction score  81097 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB


In [95]:
finalData['Experience Score']=finalData['Experience Score'].apply(lambda x: int(x))
finalData['MSISDN/Number']=finalData['MSISDN/Number'].apply(lambda x:int(x))
finalData['Engagement Score']=finalData['Engagement Score'].apply(lambda x: int(x))
finalData['Satisfaction score']=finalData['Satisfaction score'].apply(lambda x: int(x))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [96]:
finalData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81097 entries, 0 to 81096
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   MSISDN/Number       81097 non-null  int64
 1   Experience Score    81097 non-null  int64
 2   Engagement Score    81097 non-null  int64
 3   Satisfaction score  81097 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [97]:
finalData

,MSISDN/Number,Experience Score,Engagement Score,Satisfaction score
0,33601001722,111733254,516,55866885
1,33601001754,111733254,546,55866900
2,33601002511,111733254,280,55866767
3,33601007832,132541990,1209,66271599
4,33601010682,122136770,944,61068857
...,...,...,...,...
81092,33789960306,111733254,431,55866842
81093,33789967113,111733254,333,55866793
81094,33789996170,132327889,1515,66164702
81095,33789997247,111733254,330,55866792


In [98]:
finalData.to_csv('../data/finalData2.csv',index=False)